In [13]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import re, string, nltk, json, tqdm, gensim, tweepy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from pprint import pprint
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from wordcloud import WordCloud

In [14]:
# Baca token untuk akses twitter API
bearer_token = "AAAAAAAAAAAAAAAAAAAAAP8JZgEAAAAAKbnX%2B0MQauzdx2Z6mbX2FdqgMU4%3Dg9yEn4GIdcUSeClhEa3XzGZC8qYOaXi1wO8ynQZvCk2q8lrzmp"
api_key = "cY1q71v6wODsBM5MLVYLJnTCE"
api_key_secret = "NrCvpNSX5a8cDm6q7373tiecLWgif7yu6mHWL9EjYaIkvTfwX1"
access_token = "1496053841896550402-XQF7FT6ckrHvZJYejX67z5r6dn71ey"
access_token_secret = "XYrFndSg5iJjrjLAakHHSEpqBSPUqT5wBun3wAfVRfU24"

In [29]:
# Autentikasi twitter API
api = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# Kata kunci pencarian topik ini menggunakan hashtag agar lebih relevan
keyword = "#UkraineRussiaWar"
search_query = keyword + "-filter:retweets"

In [51]:
# Membuat request tweets sesuai kata kunci yang sudah ditentukan
response = tweepy.Paginator(
    api.search_recent_tweets,
    query=keyword,
    max_results=100
).flatten(limit=100)

# Menyimpan data tweets ke dalam array dan pandas dataframe dan menyimpan ke dalam file csv
tweets_array = []
for tweet in response:
    tweets_array.append(tweet.text.strip())

df_tweets = pd.DataFrame(tweets_array, columns = ['tweet'])

# Menampilkan dataframe
display(df_tweets.head())

,tweet
0,@cenkuygur #Ukraine️ #UkraineRussiaWar \nhttps...
1,RT @ukraine_world: Silence also kills. Those w...
2,♦️A clandestine call center in #Berdyansk. Sca...
3,RT @fridrian: City #Mariupol is under russian ...
4,RT @gbazov: #MARIUPOL—Interviews Part #2b—#Ukr...


In [49]:
keyword2 = "#StandWithUkraine"

100
